# Establishing Connection with the CloudSQL Postgres instance 


In [1]:
#Establishing Connection with the CloudSQL instance

import psycopg2


host = "34.85.134.242"        
database = "postgres" 
user = "postgres"        
password = "mars"      


connection = psycopg2.connect(

    host=host,
    database=database,
    user=user,
    password=password
)


cursor = connection.cursor()
cursor.execute("SELECT current_database();")
print("Connected to database:", cursor.fetchone())



Connected to database: ('postgres',)


# Tracking which Rovers need maintenance

In [2]:
cursor = connection.cursor()

query = ''' SELECT habitat_id, habitats.rover_id, status, task, charge_pct
            FROM mars_colony.habitats
            LEFT JOIN mars_colony.rovers
            ON habitats.rover_id = rovers.rover_id
            WHERE rovers.status = 'Needs Maintenance'
            ;'''

cursor.execute(query)
rows = cursor.fetchall()

for row in rows:
    print(row)


connection.commit() 

#cursor.close()

(4, 6, 'Needs Maintenance', 'Material Collection', 83.0)
(2, 10, 'Needs Maintenance', 'Transport', 66.3)


# Tracking Faulty indoor temperature sensors

In [3]:
cursor = connection.cursor()

query = ''' SELECT habitat_id, indoor_sensors.sensor_id, sensor_type, status
            FROM mars_colony.habitats
            LEFT JOIN mars_colony.indoor_sensors
            ON habitats.sensor_id = indoor_sensors.sensor_id
            WHERE indoor_sensors.status = 'Needs Maintenance' and indoor_sensors.sensor_type='Temperature'
            ;'''

cursor.execute(query)
rows = cursor.fetchall()

for row in rows:
    print(row)


connection.commit() 

#cursor.close()

(1, 1, 'Temperature', 'Needs Maintenance')
(3, 3, 'Temperature', 'Needs Maintenance')
(3, 3, 'Temperature', 'Needs Maintenance')


# Tracking dust level that is higher than 300 from environmental monitoring sensors

In [29]:
cursor = connection.cursor()

query = ''' SELECT habitat_id, dust_level, timestamp
            FROM mars_colony.habitats
            LEFT JOIN mars_colony.mars_env_monitoring
            ON habitats.record_id = mars_env_monitoring.record_id
            WHERE dust_level > 300
            ;'''

cursor.execute(query)
rows = cursor.fetchall()

for row in rows:
    print(row)


connection.commit() 

#cursor.close()

(1, 392.7, datetime.datetime(2024, 12, 9, 7, 11, 4, 293996))
(2, 334.5, datetime.datetime(2024, 12, 8, 13, 11, 4, 293996))
(4, 425.5, datetime.datetime(2024, 12, 9, 1, 11, 4, 293994))
(5, 392.7, datetime.datetime(2024, 12, 9, 7, 11, 4, 293996))
(3, 470.1, datetime.datetime(2024, 12, 10, 20, 11, 4, 293999))
(1, 497.2, datetime.datetime(2024, 12, 9, 6, 11, 4, 294005))


# Checking how much water is left in some habitats where data available

In [31]:
cursor = connection.cursor()

query = ''' SELECT habitat_id, resource_type, quantity
            FROM mars_colony.habitats
            LEFT JOIN mars_colony.resources
            ON habitats.resource_id = resources.resource_id
            WHERE resource_type = 'Water'
            ;'''

cursor.execute(query)
rows = cursor.fetchall()

for row in rows:
    print(row)


connection.commit() 

#cursor.close()

(2, 'Water', 739.6)
(4, 'Water', 725.96)
(3, 'Water', 282.76)


# Checking in which habitat the rovers for Exploration task is parked at the moment

In [34]:
cursor = connection.cursor()

query = ''' SELECT habitat_id, rovers.rover_id, task, charge_pct
            FROM mars_colony.habitats
            LEFT JOIN mars_colony.rovers
            ON habitats.rover_id = rovers.rover_id
            WHERE task = 'Exploration'
            ;'''

cursor.execute(query)
rows = cursor.fetchall()

for row in rows:
    print(row)


connection.commit() 

#cursor.close()

(2, 2, 'Exploration', 66.5)
(1, 7, 'Exploration', 66.5)


As you can see the rover #2 for exploration is parked in habitat #2 and rover #7 for exploration is parked in habitat #1

# Checking which Rovers have higher than 70 percent charge

In [37]:
cursor = connection.cursor()

query = ''' SELECT habitat_id, rovers.rover_id, status, task, charge_pct
            FROM mars_colony.habitats
            LEFT JOIN mars_colony.rovers
            ON habitats.rover_id = rovers.rover_id
            WHERE charge_pct > 70
            ;'''

cursor.execute(query)
rows = cursor.fetchall()

for row in rows:
    print(row)


connection.commit() 

#cursor.close()

(1, 1, 'Operational', 'Material Collection', 81.2)
(4, 6, 'Needs Maintenance', 'Material Collection', 83.0)
(2, 9, 'Operational', 'Material Collection', 99.0)


# Counting how many rovers each habitats have

In [67]:
cursor = connection.cursor()

query = ''' SELECT habitats.habitat_id, COUNT(rovers.rover_id)
            FROM mars_colony.habitats
            LEFT JOIN mars_colony.rovers
            ON habitats.rover_id = rovers.rover_id
            GROUP BY habitats.habitat_id
            ;'''

cursor.execute(query)
rows = cursor.fetchall()

for row in rows:
    print(row)


connection.commit() 

#cursor.close()

(3, 3)
(5, 1)
(4, 2)
(2, 3)
(1, 2)


The habitat 3 has the highest number of rovers 

# Reporting Generation

### Overall report on environment metrics near the habitats can be done as below

In [58]:
cursor = connection.cursor()

query = ''' SELECT habitat_id, radiation_level, temperature, wind_speed, dust_level
            FROM mars_colony.habitats
            LEFT JOIN mars_colony.mars_env_monitoring
            ON habitats.record_id = mars_env_monitoring.record_id
            ;'''

cursor.execute(query)
rows = cursor.fetchall()

for row in rows:
    print(row)


connection.commit() 

#cursor.close()

(1, 0.97, -17.3, 81.0, 392.7)
(2, 0.51, -51.1, 34.9, 334.5)
(3, 1.42, -56.1, 85.8, 202.3)
(4, 1.07, -45.2, 22.0, 425.5)
(4, 1.01, -26.8, 22.9, 9.0)
(5, 0.97, -17.3, 81.0, 392.7)
(2, 0.25, -0.1, 9.6, 290.3)
(3, 0.41, -1.4, 39.8, 470.1)
(2, 0.41, -51.0, 70.0, 294.9)
(3, 0.67, -18.3, 85.6, 243.4)
(1, 1.37, -28.1, 54.1, 497.2)


### Report on total accummulative quantity of food in each habitat where data is available

In [70]:
cursor = connection.cursor()

query = ''' SELECT habitats.habitat_id, SUM(resources.quantity) as total_acc_food
            FROM mars_colony.habitats
            LEFT JOIN mars_colony.resources
            ON habitats.resource_id = resources.resource_id
            WHERE resources.resource_type = 'Food'
            GROUP BY habitats.habitat_id
            ;'''

cursor.execute(query)
rows = cursor.fetchall()

for row in rows:
    print(row)


connection.commit() 

#cursor.close()

(1, 312.39)
(3, 720.8)


### Report on all the things that need maintenance

In [86]:
cursor = connection.cursor()

query = ''' SELECT habitats.habitat_id, indoor_sensors.sensor_id, indoor_sensors.status,  rovers.rover_id, rovers.status
            FROM mars_colony.habitats
            LEFT JOIN mars_colony.indoor_sensors ON habitats.sensor_id = indoor_sensors.sensor_id and indoor_sensors.status = 'Needs Maintenance'
            
            LEFT JOIN mars_colony.rovers ON habitats.rover_id = rovers.rover_id and rovers.status = 'Needs Maintenance'
            ;'''

cursor.execute(query)
rows = cursor.fetchall()

for row in rows:
    print(row)


connection.commit() 

#cursor.close()

(1, 1, 'Needs Maintenance', None, None)
(2, None, None, None, None)
(3, 3, 'Needs Maintenance', None, None)
(4, 10, 'Needs Maintenance', None, None)
(4, None, None, 6, 'Needs Maintenance')
(5, None, None, None, None)
(2, 17, 'Needs Maintenance', 10, 'Needs Maintenance')
(3, 3, 'Needs Maintenance', None, None)
(2, None, None, None, None)
(3, None, None, None, None)
(1, None, None, None, None)


# Indexing `status` column in `indoor_sensors`

In [106]:
cursor = connection.cursor()

query = ''' CREATE INDEX idx_status on mars_colony.indoor_sensors(status);'''

cursor.execute(query)

connection.commit() 

#cursor.close()

In [107]:
cursor = connection.cursor()

# PostgreSQL query to show indexes
query = '''SELECT indexname, indexdef FROM pg_indexes WHERE tablename = 'indoor_sensors';'''

cursor.execute(query)

# Fetch and print all rows containing index information
indexes = cursor.fetchall()
for index in indexes:
    print(index)

cursor.close()


('indoor_sensors_pkey', 'CREATE UNIQUE INDEX indoor_sensors_pkey ON mars_colony.indoor_sensors USING btree (sensor_id)')
('idx_status', 'CREATE INDEX idx_status ON mars_colony.indoor_sensors USING btree (status)')


# Indexing `status` column in `rovers` table

In [111]:
cursor = connection.cursor()

query = ''' CREATE INDEX rovers_status on mars_colony.rovers(status);'''

cursor.execute(query)

connection.commit() 

#cursor.close()

In [112]:
cursor = connection.cursor()

# PostgreSQL query to show indexes
query = '''SELECT indexname, indexdef FROM pg_indexes WHERE tablename = 'rovers';'''

cursor.execute(query)

# Fetch and print all rows containing index information
indexes = cursor.fetchall()
for index in indexes:
    print(index)

cursor.close()


('rovers_pkey', 'CREATE UNIQUE INDEX rovers_pkey ON mars_colony.rovers USING btree (rover_id)')
('rovers_status', 'CREATE INDEX rovers_status ON mars_colony.rovers USING btree (status)')


# indexing `dust_level` column in `mars_env_monitoring`

In [113]:
cursor = connection.cursor()

query = ''' CREATE INDEX dust_lvl on mars_colony.mars_env_monitoring(dust_level);'''

cursor.execute(query)

connection.commit() 

#cursor.close()

In [114]:
cursor = connection.cursor()

# PostgreSQL query to show indexes
query = '''SELECT indexname, indexdef FROM pg_indexes WHERE tablename = 'mars_env_monitoring';'''

cursor.execute(query)

# Fetch and print all rows containing index information
indexes = cursor.fetchall()
for index in indexes:
    print(index)

cursor.close()


('mars_env_monitoring_pkey', 'CREATE UNIQUE INDEX mars_env_monitoring_pkey ON mars_colony.mars_env_monitoring USING btree (record_id)')
('dust_lvl', 'CREATE INDEX dust_lvl ON mars_colony.mars_env_monitoring USING btree (dust_level)')
